# python利用beautifulsoup+selenium自动翻页抓取网页内容
## http://unclechen.github.io/2016/12/11/python%E5%88%A9%E7%94%A8beautifulsoup+selenium%E8%87%AA%E5%8A%A8%E7%BF%BB%E9%A1%B5%E6%8A%93%E5%8F%96%E7%BD%91%E9%A1%B5%E5%86%85%E5%AE%B9/

### Selenium Setup http://selenium-python.readthedocs.io/installation.html



## WebDriver https://sites.google.com/a/chromium.org/chromedriver/downloads
##### Extract the file and store it in a desired location.

In [ ]:
# References:
## How do I install ChromeDriver on Windows 10 and run Selenium tests with Chrome?
## https://stackoverflow.com/questions/33150351/how-do-i-install-chromedriver-on-windows-10-and-run-selenium-tests-with-chrome

### checkip: http://checkip.amazonaws.com/

### check Cookie: http://www.html-kit.com/tools/cookietester/

### check Agent: http://www.whatsmyua.info/


### check all: https://httpbin.org/get

In [60]:
import sys
import time
#from bs4 import BeautifulSoup
from selenium import webdriver
import json
import pandas as pd
import numpy as np
import requests 
from datetime import datetime, timedelta
import re

In [2]:
import sys
sys.path.insert(0,'..')
import tencent

In [3]:
import importlib
importlib.reload(tencent)

<module 'tencent' from '..\\tencent.py'>

In [15]:
driver = webdriver.Chrome(executable_path='C:\chromedriver_win32\chromedriver.exe')

In [16]:
driver.get('http://www.html-kit.com/tools/cookietester/')

In [17]:
driver.execute_script("window.open('http://www.html-kit.com/tools/cookietester/', 'new_window')")

In [62]:
def whiteip(ip):
    #myip = requests.get('http://ip.42.pl/raw').text
    myip=ip
    print("public ip:", myip)
    addwhitelist = requests.get('http://web.http.cnapi.cc/index/index/save_white?neek=50451&appkey=7383395d5db3dcdf582cb4d074bd5bdc&white=' + myip).text
    print(addwhitelist)

public ip: 111.63.3.189
{"code":115,"success":false,"msg":"该ip已经在您的白名单中！","data":[]}


In [67]:
# proxy pool
#
# https://cuiqingcai.com/5094.html
# http://h.zhimaruanjian.com/newrecharge/
# 需要先设置IP白名单： http://h.zhimaruanjian.com/wirte_list/#recharge
# 白名单API： web.http.cnapi.cc/index/index/save_white?neek=50451&appkey=7383395d5db3dcdf582cb4d074bd5bdc&white=您的ip
# http://h.zhimaruanjian.com/newrecharge/

test_url = 'https://www.baidu.com/'
timeout = 60


def get_proxy_zhima():
    proxy_api = 'http://webapi.http.zhimacangku.com/getip?num=1&type=2&pro=&city=0&yys=0&port=1&pack=27521&ts=0&ys=0&cs=0&lb=1&sb=0&pb=4&mr=1&regions='
    resp = requests.get(url=proxy_api)
    data = resp.json()
#     print(data)   
#     driver.get("http://webapi.http.zhimacangku.com/getip?num=1&type=2&pro=&city=0&yys=0&port=1&pack=27521&ts=0&ys=0&cs=0&lb=1&sb=0&pb=4&mr=1&regions=")
#     #  {"code":0,"success":true,"msg":"0","data":[{"ip":"112.116.20.116","port":2136}]}
#     dict_from_json = json.loads(driver.find_element_by_tag_name('body').text)
    
    if(data['success'] == True):
        proxy = (data['data'][0]['ip'] + ':' + str( data['data'][0]['port']))
        print("proxy: ", proxy)
    else:
        print(data)
        if(data['code'] == 113): 
            ip = re.findall( r'[0-9]+(?:\.[0-9]+){3}', data['msg'] )
            print('need set white ip list...', ip)
            white_ip=ip[0]
            whiteip(white_ip)
            proxy=get_proxy_zhima()
    return proxy

def test_proxy(proxy):
    try:
        proxies = {
            'https': 'http://' + proxy
        }
        start_time = time.time()
        requests.get(test_url, timeout=timeout, proxies=proxies, headers={'Connection':'close'})
        end_time = time.time()
        used_time = end_time - start_time
        print('Proxy Valid', 'Used Time:', used_time)
        return True, used_time
    except Exception as e: 
        print(e)
        return False, None
# TEST   
# proxy = get_proxy_zhima(driver)
# print('Testing proxy', proxy)
# time.sleep(1)
# test_flag, test_result = test_proxy(proxy=proxy)


In [36]:
from fake_useragent import UserAgent


'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36'

In [71]:
#PROXY = "23.23.23.23:3128" # IP:PORT or HOST:PORT
PROXY=get_proxy_zhima()

chrome_options = webdriver.ChromeOptions()
# switch Proxy

print('new agent:', UserAgent().random)
chrome_options.add_argument('--proxy-server=%s' % PROXY)
# switch Agent
chrome_options.add_argument("user-agent=%s"% UserAgent().random)
chrome = webdriver.Chrome(chrome_options=chrome_options,executable_path='C:\chromedriver_win32\chromedriver.exe')
chrome.get("http://httpbin.org/get")

proxy:  116.53.197.104:4278
new agent: Mozilla/5.0 (Windows NT 5.1; rv:21.0) Gecko/20100101 Firefox/21.0


In [9]:
chrome.execute_script("window.open('http://google.com', 'new_window')")

In [13]:
chrome.switch_to_window(chrome.window_handles[1])

In [33]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(webdriver,xpath):
    try:
        webdriver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#check_exists_by_xpath(driver,'//img[@id="captchaImg111"]' )

In [44]:
#https://stackoverflow.com/questions/36636255/selenium-downloading-different-captcha-image-than-the-one-in-browser
from PIL import Image
def get_captcha(driver, element, path):
    # now that we have the preliminary stuff out of the way time to get that image :D
    location = element.location
    size = element.size
    # saves screenshot of entire page
    driver.save_screenshot(path)

    # uses PIL library to open image in memory
    image = Image.open(path)

    left = location['x']
    top = location['y'] #+ 140
    right = location['x'] + size['width']
    bottom = location['y'] + size['height'] #+ 140

    image = image.crop((left, top, right, bottom))  # defines crop points
    image.save(path, 'png')  # saves new cropped image
    return image

#get_captcha(driver, ele, 'test1.png')

In [161]:
def query_index():
    url = 'http://www.tianjin-air.com/'
    driver.get(url)
    #get session
    driver.execute_script('$("#airplaneCity1").val("天津");  $("#airplaneCity1_value").val("TSN");  $("#airplaneCity2").val("青岛");  $("#airplaneCity2_value").val("TAO");  $("#goday").val("2018-08-30");  $("input[name=\'tripType\']").val("OW");  $("input[name=\'flexibleSearch\']").val("true");  $("#adult").val("1");  $("#child").val("1");  $("#infant").val("0");  $("#pwd").val("0");  $("#mwd").val("0");  $("#cabinType").val("Economy");  $("#searchflight").attr("action", "/flight/AirLowFareSearch");  $("#searchflight").submit();')
    fix_capcha(driver)

In [361]:
def fix_capcha(driver):
    i = 0
    while(check_exists_by_xpath(driver,'//img[@id="captchaImg"]')):
        try:
            #driver.find_element_by_xpath('//img[@id="captchaImg"]').click()
            driver.execute_script('$("#captchaImg").trigger("click")')
            time.sleep(1)
            print ("==========GOT CAPCHA ===   "+str(i)+"  ===========")
            ele= driver.find_element_by_xpath('//img[@id="captchaImg"]')
            captha = get_captcha(driver, ele, 'test1.png')
            try:
                res = tencent.pil_ocr(captha)
            except requests.exceptions.RequestException as e:
                continue
            if(res['msg']== 'ok' and len(res["data"]["item_list"][0]["itemstring"]) == 4):
                ocr = res["data"]["item_list"][0]["itemstring"]
                script='$("#imgCaptchaValue").val("'+ ocr +'"); '
            else:
                print("OCR ERROR......")
                continue
            #print ("=====SCRIPT======  " , script)
            driver.execute_script(script)
            i+=1
            time.sleep(2)
            driver.find_element_by_xpath("//div[contains(@class, 'hna-loading')]//button").click()
            time.sleep(3)
        except:
            continue
    if (i==0):
        print("No Captha")
    else:
        print("capcha fixed")
    return True
#fix_capcha(driver)

In [330]:
#date compare in python
#pip install python-dateutil
from dateutil import parser as dateparser
def compare_date(str_dt1,str_dt2):
    dt1 = dateparser.parse(str_dt1)
    dt2 = dateparser.parse(str_dt2)
    return (dt1-dt2).days
#compare_date("2018-12-04","2018-11-22")

In [329]:
from dateutil import parser as dateparser

def getMonthJson():
    driver.get("http://www.tianjin-air.com/flight/getPriceTrend")
    dict_from_json = json.loads(driver.find_element_by_tag_name('body').text)

    if(dict_from_json['code'] == 'fail'):
        query_index()


    #get Json to Pandas
    go_date=[]
    go_pric=[]
    bk_date=[]
    bk_pric=[]

    for i,godata in enumerate(dict_from_json['data'][0]['fares']):
        go_pric.append(godata['amount'])
        go_date.append(godata['date'])

    for i,godata in enumerate(dict_from_json['data'][1]['fares']):
        bk_pric.append(godata['amount'])
        bk_date.append(godata['date'])

    flight_list = pd.DataFrame(
        {'go_date': go_date,
         'go_pric': go_pric,
         'bk_date': bk_date,
         'bk_pric': bk_pric,
        })

    flight_list['go_pric']=flight_list['go_pric'].replace(0,100000)
    flight_list['bk_pric']=flight_list['bk_pric'].replace(0,100000)

    sum_prics=[]
    godates=[]
    bkdates=[]
    durations=[]
    fromnows=[]
    
    for i, gopric in enumerate(flight_list["go_pric"].tolist()):
        for j,bkpric in enumerate(flight_list["bk_pric"].tolist()):
            sum_pric = gopric + bkpric
            godate = flight_list["go_date"][i]
            bkdate = flight_list["bk_date"][j]
            duration = ( parser.parse(bkdate) - parser.parse(godate)).days
            fromnow = (parser.parse(godate) -  datetime.now()  ).days
            sum_prics.append(sum_pric)
            godates.append(parser.parse(godate))
            bkdates.append(parser.parse(bkdate))
            durations.append(duration)
            fromnows.append(fromnow)
    adj_list = pd.DataFrame(
        {'sum_pric': sum_prics,
         'godates': godates,
         'bkdates': bkdates,
         'durations':durations,
         'fromnows':fromnows
        })

    sort_list = adj_list.sort_values(by=['sum_pric'])
    filter_list = sort_list[sort_list['bkdates'] > sort_list['godates']]
    return filter_list

In [280]:

# Find Ele by MAX TEXT
# Notice1: find_elements_by_xpath    "elements" not "element"
# lambda
def click_min_price(back=1):
    t1 = driver.find_elements_by_xpath("//div[@class='flight-search-group' and @data-index='"+str(back)+"' and //li[contains(@class,'active')]]//label[@class='hna-radio']")
    max_element = min(t1, key=lambda e: e.get_attribute('innerHTML'))
    max_element.click()

#click_min_price(1) #0: go   1:back

In [344]:
def add_days(base, add):
    base = dateparser.parse(base)
    #today = base.strftime('%Y-%m-%d')
    target = base + timedelta(days=add)
    return target.strftime('%Y-%m-%d')
# add_days(datetime.today(), 10)

In [311]:
#check if the “不适用” in go & back tabs?
def bushiyong(webdriver):
    try:
        webdriver.find_element_by_xpath("//div[@class='flight-search-group']//li[contains(@class,'active')]//strong[contains(.,'不适用')]")
        try:
            # go date bushiyong
            driver.find_elements_by_xpath("//div[@class='flight-search-group'][1]//li[contains(@class,'active')]")
            return 0
        except:
            # bk date bushiyong
            return 1
    except NoSuchElementException:
        return -1
#active_bushiyong(driver)

In [338]:
# Cookie Tester:
# http://www.html-kit.com/tools/cookietester/

'2335.00'

In [26]:
def testfn():
    z=0
    while (True):
        while z<100:
            z+=1
            print(z)
            if(z>30):
                raise StopIteration
                print('Exit')
                return None
testfn()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
Exit


In [27]:
class StopIteration(Exception): pass

In [352]:
def query_detail(driver, detination_code, detination_name, godate):
    print("start time:", datetime.now())
    record_count = 0
    i=0
    while i<75:
        j=1
        while j<15:
            bkdate = add_days(godate , j)
            print("godate: ",godate,"   -------bkdate: ",bkdate)
            driver.execute_script('$("#airplaneCity1").val("天津");  $("#airplaneCity1_value").val("TSN");  $("#airplaneCity2").val("'+detination_name+'");  $("#airplaneCity2_value").val("'+detination_code+'");  $("#goday").val("'+godate+'");  $("#backday").val("'+bkdate+'");   $("input[name=\'tripType\']").val("RT");  $("input[name=\'flexibleSearch\']").val("true");  $("#adult").val("1");  $("#child").val("1");  $("#infant").val("0");  $("#pwd").val("0");  $("#mwd").val("0");  $("#cabinType").val("Economy");  $("#searchflight").attr("action", "/flight/AirLowFareSearch");  $("#searchflight").submit();')
            time.sleep(1)
            fix_capcha(driver)
            bsy = bushiyong(driver)
            if(bsy < 0):
                try:
                    time.sleep(1)
                    click_min_price(0)
                    time.sleep(1)
                    fix_capcha(driver)
                    time.sleep(1)
                    click_min_price(1)
                    time.sleep(3)
                    fix_capcha(driver)
                    time.sleep(1)
                    price=driver.find_element_by_xpath("//label[@id='priceT']").get_attribute('innerHTML')
                    record_count +=1
                    print("=============record_count:", record_count ,"============ price:",price,"==================")
                    pass # todo: record
                except:
                    print("ERROR OCCERED......")
                    continue
            elif(bsy == 0):
                #godate not work
                #godate + 1
                print("godate not work...  switch to next back date...")
                time.sleep(3)
                break
            else:
                #back date not work
                print("back date not work... switch to next back date...")
                time.sleep(3)
                #bkdate + 1
                continue
            j+=1
        godate = add_days(godate , 1)
        i+=1
#query_detail(driver,TAO, "青岛","2018-08-30")

In [ ]:
def query_detail_wProxySwitch(driver, detination_code, detination_name, godate):
    print("start time:", datetime.now())
    record_count = 0
    i=0
    while i<75:
        j=1
        while j<15:
            bkdate = add_days(godate , j)
            print("godate: ",godate,"   -------bkdate: ",bkdate)
            driver.execute_script('$("#airplaneCity1").val("天津");  $("#airplaneCity1_value").val("TSN");  $("#airplaneCity2").val("'+detination_name+'");  $("#airplaneCity2_value").val("'+detination_code+'");  $("#goday").val("'+godate+'");  $("#backday").val("'+bkdate+'");   $("input[name=\'tripType\']").val("RT");  $("input[name=\'flexibleSearch\']").val("true");  $("#adult").val("1");  $("#child").val("1");  $("#infant").val("0");  $("#pwd").val("0");  $("#mwd").val("0");  $("#cabinType").val("Economy");  $("#searchflight").attr("action", "/flight/AirLowFareSearch");  $("#searchflight").submit();')
            time.sleep(1)
            fix_capcha(driver)
            bsy = bushiyong(driver)
            if(bsy < 0):
                try:
                    time.sleep(1)
                    click_min_price(0)
                    time.sleep(1)
                    fix_capcha(driver)
                    time.sleep(1)
                    click_min_price(1)
                    time.sleep(3)
                    fix_capcha(driver)
                    time.sleep(1)
                    price=driver.find_element_by_xpath("//label[@id='priceT']").get_attribute('innerHTML')
                    record_count +=1
                    print("=============record_count:", record_count ,"============ price:",price,"==================")
                    pass # todo: record
                except:
                    print("ERROR OCCERED......")
                    continue
            elif(bsy == 0):
                #godate not work
                #godate + 1
                print("godate not work...  switch to next back date...")
                time.sleep(3)
                break
            else:
                #back date not work
                print("back date not work... switch to next back date...")
                time.sleep(3)
                #bkdate + 1
                continue
            j+=1
        godate = add_days(godate , 1)
        i+=1
#query_detail(driver,TAO, "青岛","2018-08-30")

In [ ]:
#             # SWITCH PROXY HERE!
#             raise StopIteration
#             # if time > 20min
#                 # update i & j
#                 # switch proxy(driver, )
#         except StopIteration: 
#             break 


In [342]:
query_index()
#goto AirFareFamiliesFlexibleForward

No Captha


In [362]:
query_detail(driver,'TAO', "青岛","2018-08-30")

start time: 2018-08-19 21:34:20.825387
godate:  2018-08-30    -------bkdate:  2018-08-31
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '总或示能', 'itemcoord': [{'x': 3, 'y': 4, 'width': 89, 'height': 22}], 'words': [{'character': '总', 'confidence': 0.305605}, {'character': '或', 'confidence': 0.484473}, {'character': '示', 'confidence': 0.995561}, {'character': '能', 'confidence': 0.481394}]}]}}
==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '之处及', 'itemcoord': [{'x': 4, 'y': 5, 'width': 89, 'height': 21}], 'words': [{'character': '之', 'confidence': 0.939298}, {'character': '处', 'confidence': 0.999803}, {'character': '及', 'confidence': 0.999871}]}]}}
OCR ERROR......
==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '外能规构', 'itemcoord': [{'x': 3, 'y'

No Captha
No Captha
No Captha
=============record_count: 10 ============ price: 1890.00 ==================
godate:  2018-08-31    -------bkdate:  2018-09-11
No Captha
No Captha
No Captha
=============record_count: 11 ============ price: 1950.00 ==================
godate:  2018-08-31    -------bkdate:  2018-09-12
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '出取示旅', 'itemcoord': [{'x': 7, 'y': 8, 'width': 82, 'height': 20}], 'words': [{'character': '出', 'confidence': 0.998142}, {'character': '取', 'confidence': 0.982255}, {'character': '示', 'confidence': 0.999897}, {'character': '旅', 'confidence': 0.965738}]}]}}
capcha fixed
No Captha
No Captha
=============record_count: 12 ============ price: 1890.00 ==================
godate:  2018-08-31    -------bkdate:  2018-09-13
No Captha
No Captha
No Captha
=============record_count: 13 ============ price: 1890.00 ==================
godate:  2018-08-31    -------b

res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '之要做的', 'itemcoord': [{'x': 5, 'y': 4, 'width': 83, 'height': 21}], 'words': [{'character': '之', 'confidence': 0.996482}, {'character': '要', 'confidence': 0.328482}, {'character': '做', 'confidence': 0.48844}, {'character': '的', 'confidence': 0.719514}]}]}}
==========GOT CAPCHA ===   2  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '文奏天则', 'itemcoord': [{'x': 4, 'y': 8, 'width': 86, 'height': 21}], 'words': [{'character': '文', 'confidence': 0.446968}, {'character': '奏', 'confidence': 0.726255}, {'character': '天', 'confidence': 0.960824}, {'character': '则', 'confidence': 0.997475}]}]}}
==========GOT CAPCHA ===   3  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '责为降够', 'itemcoord': [{'x': 4, 'y': 7, 'width': 88, 'height': 20}], 'words': [{'character': '责', 'confidence': 0.994764}, {'character': '

No Captha
=============record_count: 31 ============ price: 1890.00 ==================
godate:  2018-09-02    -------bkdate:  2018-09-06
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '人退试的', 'itemcoord': [{'x': 9, 'y': 7, 'width': 82, 'height': 23}], 'words': [{'character': '人', 'confidence': 0.99677}, {'character': '退', 'confidence': 0.392714}, {'character': '试', 'confidence': 0.760066}, {'character': '的', 'confidence': 0.973011}]}]}}
==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '超诊信专', 'itemcoord': [{'x': 3, 'y': 4, 'width': 92, 'height': 21}], 'words': [{'character': '超', 'confidence': 0.761437}, {'character': '诊', 'confidence': 0.952905}, {'character': '信', 'confidence': 0.991376}, {'character': '专', 'confidence': 0.679004}]}]}}
==========GOT CAPCHA ===   2  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': 

==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '神处款医', 'itemcoord': [{'x': 2, 'y': 4, 'width': 93, 'height': 21}], 'words': [{'character': '神', 'confidence': 0.999936}, {'character': '处', 'confidence': 0.999696}, {'character': '款', 'confidence': 0.999391}, {'character': '医', 'confidence': 0.997494}]}]}}
capcha fixed
No Captha
No Captha
=============record_count: 40 ============ price: 2030.00 ==================
godate:  2018-09-02    -------bkdate:  2018-09-15
No Captha
No Captha
No Captha
=============record_count: 41 ============ price: 1890.00 ==================
godate:  2018-09-02    -------bkdate:  2018-09-16
No Captha
No Captha
No Captha
=============record_count: 42 ============ price: 1950.00 ==================
godate:  2018-09-03    -------bkdate:  2018-09-04
No Captha
No Captha
No Captha
=============record_count: 43 ============ price: 2375.00 ==================
godate:  2018-09-03    -------b

...

No Captha
=============record_count: 976 ============ price: 2990.00 ==================
godate:  2018-11-08    -------bkdate:  2018-11-19
No Captha
No Captha
No Captha
=============record_count: 977 ============ price: 2990.00 ==================
godate:  2018-11-08    -------bkdate:  2018-11-20
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '姓都眠象', 'itemcoord': [{'x': 5, 'y': 7, 'width': 90, 'height': 22}], 'words': [{'character': '姓', 'confidence': 0.960922}, {'character': '都', 'confidence': 0.606489}, {'character': '眠', 'confidence': 0.620643}, {'character': '象', 'confidence': 0.806203}]}]}}
==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '续代持', 'itemcoord': [{'x': 7, 'y': 8, 'width': 81, 'height': 21}], 'words': [{'character': '续', 'confidence': 0.716401}, {'character': '代', 'confidence': 0.862964}, {'character': '持', 'c

No Captha
=============record_count: 986 ============ price: 2990.00 ==================
godate:  2018-11-09    -------bkdate:  2018-11-16
No Captha
No Captha
No Captha
=============record_count: 987 ============ price: 2990.00 ==================
godate:  2018-11-09    -------bkdate:  2018-11-17
No Captha
No Captha
No Captha
=============record_count: 988 ============ price: 2990.00 ==================
godate:  2018-11-09    -------bkdate:  2018-11-18
No Captha
No Captha
No Captha
=============record_count: 989 ============ price: 2990.00 ==================
godate:  2018-11-09    -------bkdate:  2018-11-19
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '则象加0', 'itemcoord': [{'x': 4, 'y': 4, 'width': 86, 'height': 21}], 'words': [{'character': '则', 'confidence': 0.99986}, {'character': '象', 'confidence': 0.927791}, {'character': '加', 'confidence': 0.967062}, {'character': '0', 'confidence': 0.726609}]}]}}
=

==========GOT CAPCHA ===   3  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '传以必康', 'itemcoord': [{'x': 6, 'y': 5, 'width': 83, 'height': 20}], 'words': [{'character': '传', 'confidence': 0.987899}, {'character': '以', 'confidence': 0.999738}, {'character': '必', 'confidence': 0.98351}, {'character': '康', 'confidence': 0.966556}]}]}}
==========GOT CAPCHA ===   4  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '盖要长老', 'itemcoord': [{'x': 4, 'y': 6, 'width': 92, 'height': 22}], 'words': [{'character': '盖', 'confidence': 0.940327}, {'character': '要', 'confidence': 0.999758}, {'character': '长', 'confidence': 0.999987}, {'character': '老', 'confidence': 0.977293}]}]}}
==========GOT CAPCHA ===   5  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '传员符策', 'itemcoord': [{'x': 4, 'y': 3, 'width': 88, 'height': 22}], 'words': [{'character': '传'

==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '者诊请脑', 'itemcoord': [{'x': 5, 'y': 5, 'width': 85, 'height': 26}], 'words': [{'character': '者', 'confidence': 0.331841}, {'character': '诊', 'confidence': 0.531282}, {'character': '请', 'confidence': 0.909994}, {'character': '脑', 'confidence': 0.383336}]}]}}
==========GOT CAPCHA ===   2  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '手上次论', 'itemcoord': [{'x': 4, 'y': 3, 'width': 91, 'height': 21}], 'words': [{'character': '手', 'confidence': 0.241959}, {'character': '上', 'confidence': 0.983665}, {'character': '次', 'confidence': 0.858852}, {'character': '论', 'confidence': 0.22868}]}]}}
==========GOT CAPCHA ===   3  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '退法属我', 'itemcoord': [{'x': 5, 'y': 7, 'width': 86, 'height': 24}], 'words': [{'character': '退'

No Captha
No Captha
No Captha
=============record_count: 1031 ============ price: 2990.00 ==================
godate:  2018-11-12    -------bkdate:  2018-11-22
No Captha
No Captha
No Captha
=============record_count: 1032 ============ price: 2990.00 ==================
godate:  2018-11-12    -------bkdate:  2018-11-23
No Captha
No Captha
No Captha
=============record_count: 1033 ============ price: 2990.00 ==================
godate:  2018-11-12    -------bkdate:  2018-11-24
==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '站用无构', 'itemcoord': [{'x': 4, 'y': 4, 'width': 89, 'height': 21}], 'words': [{'character': '站', 'confidence': 0.945018}, {'character': '用', 'confidence': 0.501922}, {'character': '无', 'confidence': 0.987444}, {'character': '构', 'confidence': 0.999671}]}]}}
capcha fixed
No Captha
No Captha
=============record_count: 1034 ============ price: 2990.00 ==================
godate:  2018-11-12    

In [350]:
t1 = driver.find_elements_by_xpath("//div[@class='flight-search-group' and @data-index='0' and //li[contains(@class,'active')]]//label[@class='hna-radio']")

In [351]:
max_element = min(t1, key=lambda e: e.get_attribute('innerHTML'))

In [253]:
back=1
last_xpath_str = "//div[@class='flight-search-group' and @data-index='"+ str(back) +"' and //li[contains(@class,'active')]]//ul[@class='flight-search-calendar']//li[last()]"
active_xpath_str = "//div[@class='flight-search-group' and @data-index='"+ str(back) +"']//li[contains(@class,'active')][1]"
last = driver.find_element_by_xpath(last_xpath_str)
active = driver.find_element_by_xpath(active_xpath_str)
last == active

True

In [249]:
active

<selenium.webdriver.remote.webelement.WebElement (session="a34bed29e9f624184a89c38f3046d029", element="0.9275905355183722-1")>

In [260]:
# def click_next_day():
#     #clicknext()
#     pass
# def click_day_x(x):
#     if(x=='next'):
#         click_next_day()
#     elif(x=="1st"):
#         click(1) # to be done

# click next date TAB in AirFareFamiliesFlexibleForward
def next_day_click(back=0):
    last_xpath_str = "//div[@class='flight-search-group' and @data-index='"+ str(back) +"' and //li[contains(@class,'active')]]//ul[@class='flight-search-calendar']//li[last()]"
    active_xpath_str = "//div[@class='flight-search-group' and @data-index='"+ str(back) +"']//li[contains(@class,'active')][1]"
    last = driver.find_element_by_xpath(last_xpath_str)
    active = driver.find_element_by_xpath(active_xpath_str)
    if(last == active):
        print("current is the last one...")
        next_btn = "//div[@class='flight-search-group' and @data-index='"+ str(back) +"' and //li[contains(@class,'active')]]//div[contains(@class,'next-week')]"
        # Click > btn.
        driver.find_element_by_xpath(next_btn).click()
        time.sleep(3)
        
        first_day_of_week = "//div[@class='flight-search-group' and @data-index='1' and //li[contains(@class,'active')]]//ul[@class='flight-search-calendar']//li[1]"
        #click 1st TAB.
        driver.find_element_by_xpath(first_day_of_week).click()
        
    else:
        print("to be done")
        #click BTN next to .active
        
        
        #deal 不适用

In [261]:
next_day_click(1)

current is the last one...


In [210]:
1. get into AirFareFamiliesFlexibleForward
2. get into 


check if the active TAB is the last flight-search-date TAB

$x



In [237]:
def main():
    #get session
    driver.execute_script('$("#airplaneCity1").val("天津");  $("#airplaneCity1_value").val("TSN");  $("#airplaneCity2").val("青岛");  $("#airplaneCity2_value").val("TAO");  $("#goday").val("2018-08-30");  $("input[name=\'tripType\']").val("OW");  $("input[name=\'flexibleSearch\']").val("true");  $("#adult").val("1");  $("#child").val("1");  $("#infant").val("0");  $("#pwd").val("0");  $("#mwd").val("0");  $("#cabinType").val("Economy");  $("#searchflight").attr("action", "/flight/AirLowFareSearch");  $("#searchflight").submit();')
    sortdf = getMonthJson()
    min_df = sortdf[sortdf['sum_pric'] < min(sortdf['sum_pric']) +100]

    min_df['durations_rank'] =  1- (abs( min_df['durations']-3 )/15 )
    min_df['fromnows_rank'] = (min_df['fromnows']/30)
    # min_df['rank']=min_df['sum_pric'] 

    maxprice = max(min_df['sum_pric'])

    min_df['price_rank'] = maxprice *10 -min_df['sum_pric'] 
    min_df['rank'] = min_df['price_rank'] *min_df['durations_rank']  *min_df['fromnows_rank']
    sort_df = min_df.sort_values(by=['rank'], ascending=False)
    sort_df

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,bkdates,durations,fromnows,godates,sum_pric,durations_rank,fromnows_rank,price_rank,rank
809,2018-09-15,3,25,2018-09-12,825,1.000000,0.833333,8075,6729.166667
839,2018-09-15,2,26,2018-09-13,825,0.933333,0.866667,8075,6531.777778
777,2018-09-13,2,24,2018-09-11,825,0.933333,0.800000,8075,6029.333333
779,2018-09-15,4,24,2018-09-11,825,0.933333,0.800000,8075,6029.333333
716,2018-09-12,3,22,2018-09-09,825,1.000000,0.733333,8075,5921.666667
807,2018-09-13,1,25,2018-09-12,825,0.866667,0.833333,8075,5831.944444
776,2018-09-12,1,24,2018-09-11,825,0.866667,0.800000,8075,5598.666667
717,2018-09-13,4,22,2018-09-09,825,0.933333,0.733333,8075,5526.888889
715,2018-09-11,2,22,2018-09-09,825,0.933333,0.733333,8075,5526.888889
714,2018-09-10,1,22,2018-09-09,825,0.866667,0.733333,8075,5132.111111


In [216]:
sortdf.iloc[0]['sum_pric']

825

In [181]:
dfx = pd.DataFrame( np.random.randn(30,3), columns = ['a','b','c'])
df_filtered = dfx.query('a>0').query('0<b<2')

In [183]:
dfx

,a,b,c
0,0.112528,-1.722419,-1.945103
1,1.435195,-1.283366,-0.329223
2,0.627067,1.410977,-0.661865
3,1.202640,-0.339426,-0.052876
4,-1.186670,-0.613272,-0.761086
5,-1.038746,1.128630,1.717600
6,0.203765,1.001837,-0.026989
7,0.969150,-0.272630,0.768486
8,1.755751,-0.728999,-0.909045
9,1.088469,-0.505566,0.366574


In [133]:
soup.find("body").text

'{"data":[{"dst":"TAO","dstName":"青岛","fares":[{"amount":0,"currency":"CNY","date":"2018-08-17"},{"amount":945,"currency":"CNY","date":"2018-08-18"},{"amount":390,"currency":"CNY","date":"2018-08-19"},{"amount":560,"currency":"CNY","date":"2018-08-20"},{"amount":390,"currency":"CNY","date":"2018-08-21"},{"amount":390,"currency":"CNY","date":"2018-08-22"},{"amount":720,"currency":"CNY","date":"2018-08-23"},{"amount":500,"currency":"CNY","date":"2018-08-24"},{"amount":945,"currency":"CNY","date":"2018-08-25"},{"amount":450,"currency":"CNY","date":"2018-08-26"},{"amount":390,"currency":"CNY","date":"2018-08-27"},{"amount":450,"currency":"CNY","date":"2018-08-28"},{"amount":390,"currency":"CNY","date":"2018-08-29"},{"amount":450,"currency":"CNY","date":"2018-08-30"},{"amount":390,"currency":"CNY","date":"2018-08-31"},{"amount":640,"currency":"CNY","date":"2018-09-01"},{"amount":275,"currency":"CNY","date":"2018-09-02"},{"amount":450,"currency":"CNY","date":"2018-09-03"},{"amount":325,"curr

In [131]:
dict_from_json

{'code': 'success',
 'data': [{'dst': 'TAO',
   'dstName': '青岛',
   'fares': [{'amount': 0, 'currency': 'CNY', 'date': '2018-08-17'},
    {'amount': 945, 'currency': 'CNY', 'date': '2018-08-18'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-19'},
    {'amount': 560, 'currency': 'CNY', 'date': '2018-08-20'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-21'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-22'},
    {'amount': 720, 'currency': 'CNY', 'date': '2018-08-23'},
    {'amount': 500, 'currency': 'CNY', 'date': '2018-08-24'},
    {'amount': 945, 'currency': 'CNY', 'date': '2018-08-25'},
    {'amount': 450, 'currency': 'CNY', 'date': '2018-08-26'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-27'},
    {'amount': 450, 'currency': 'CNY', 'date': '2018-08-28'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-29'},
    {'amount': 450, 'currency': 'CNY', 'date': '2018-08-30'},
    {'amount': 390, 'currency': 'CNY', 'date': '2018-08-31'}

In [ ]:
http://www.tianjin-air.com/flight/AirFareFamiliesFlexibleForward

==========GOT CAPCHA ===   0  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '出底治徐', 'itemcoord': [{'x': 6, 'y': 5, 'width': 86, 'height': 20}], 'words': [{'character': '出', 'confidence': 0.914388}, {'character': '底', 'confidence': 0.974306}, {'character': '治', 'confidence': 0.357021}, {'character': '徐', 'confidence': 0.977761}]}]}}
==========GOT CAPCHA ===   1  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '顽系备康', 'itemcoord': [{'x': 3, 'y': 4, 'width': 90, 'height': 21}], 'words': [{'character': '顽', 'confidence': 0.762222}, {'character': '系', 'confidence': 0.999984}, {'character': '备', 'confidence': 0.995317}, {'character': '康', 'confidence': 0.539247}]}]}}
==========GOT CAPCHA ===   2  ===========
res---------> {'ret': 0, 'msg': 'ok', 'data': {'item_list': [{'item': '', 'itemstring': '向件改元', 'itemcoord': [{'x': 4, 'y': 5, 'width': 83, 'height': 21}], 'words': [{'character': '向

In [5]:
type(driver.page_source)

str

In [15]:
while page < 3: # 共50页，这里是手工指定的
	soup = BeautifulSoup(driver.page_source, "html.parser")
	current_names = soup.select('div.ranking-table') # 选择器用ranking-table css class，可以取出包含本周、上周的两个table的div标签
	for current_name_list in current_names:
		# print current_name_list['data-cat']
		if current_name_list['data-cat'] == 'thisWeek': # 这次我只想抓取本周，如果想抓上周，改一下这里为lastWeek即可
			names = current_name_list.select('td.star-name > a') # beautifulsoup选择器语法
			counter = 0;
			for star_name in names:
				counter = counter + 1;
				print (star_name.text) # 明星的名字是a标签里面的文本，虽然a标签下面除了文本还有一个与文本同级别的img标签，但是.text输出的只是文本而已
				name_counter = name_counter + 1;
	driver.find_element_by_xpath("//a[contains(text(),'下一页')]").click() # selenium的xpath用法，找到包含“下一页”的a标签去点击
	page = page + 1
	time.sleep(2) # 睡2秒让网页加载完再去读它的html代码

print (name_counter) # 共爬取得明星的名字数量
driver.quit()

王俊凯
马天宇
金秀贤
曾艳芬
易烊千玺
李学政
黄子韬
郑智薰
迪丽热巴
周笔畅
黄婷婷
金钟国
陈伟霆
史文翔
余枫
鹿晗
华晨宇
钟汉良
陈乔恩
李易峰
张国荣
张艺兴
刘昊然
胡宇威
SNH48
蔡徐坤
胡一天
朴施厚
王凯
杨洋
赵丽颖
李丞峰
张丹峰
王源
山下智久
黄致列
白敬亭
樱井翔
许魏洲
欧豪
胡歌
郑爽
黄景瑜
宋茜
宋威龙
迪·杰西达邦·福尔迪
徐海乔
马可
杨培安
王嘉尔
吴磊
朱镇模
吴亦凡
薛之谦
杨宗纬
杨幂
冯建宇
林青霞
周芷莹
乔任梁
61
